In [1]:
from dotenv import load_dotenv
import os
from IPython.display import display, Markdown

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

### Logical Routing

In [38]:
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
import pprint

# Data model
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["python_docs", "js_docs", "golang_docs"] = Field(
        ..., 
        description="Given a user question choose which datasource would be most relevant for answering their question",
    )

# LLM with function call 
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
structured_llm = llm.with_structured_output(RouteQuery)

# Prompt 
template = """You are an expert at routing a user question to the appropriate data source.

Based on the programming language the question is referring to, route it to the relevant data source.

User Question: {question}"""

prompt = ChatPromptTemplate.from_template(template)

# system = """You are an expert at routing a user question to the appropriate data source.

# Based on the programming language the question is referring to, route it to the relevant data source."""

# prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", system),
#         ("human", "{question}"),
#     ]
# )

# Define router 
router = prompt | structured_llm

C:\Users\Sorou\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_openai\chat_models\base.py:1375: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo-0125 since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


In [51]:
question = """Why doesn't the following code work:

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(["human", "speak in {language}"]) "])
prompt.invoke("french")
"""

result = router.invoke(question)
pprint.pprint(result)

RouteQuery(datasource='python_docs')


In [47]:
def choose_route(result):
    if "python_docs" in result.datasource.lower():
        ### Logic here 
        return "chain for python_docs"
    elif "js_docs" in result.datasource.lower():
        ### Logic here 
        return "chain for js_docs"
    else:
        ### Logic here 
        return "golang_docs"

from langchain_core.runnables import RunnableLambda

full_chain = router | RunnableLambda(choose_route)

In [48]:
full_chain.invoke({"question": question})

'chain for python_docs'

### Semantic Routing

In [52]:
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# Two prompts
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{query}"""

math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{query}"""

# Embed prompts
embeddings = OpenAIEmbeddings()
prompt_templates = [physics_template, math_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)

# Route question to prompt 
def prompt_router(input):
    # Embed question
    query_embedding = embeddings.embed_query(input["query"])
    # Compute similarity
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    # Chosen prompt 
    print("Using MATH" if most_similar == math_template else "Using PHYSICS")
    return PromptTemplate.from_template(most_similar)


chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | ChatOpenAI()
    | StrOutputParser()
)

print(chain.invoke("What's a black hole"))

Using PHYSICS
A black hole is a region in space where gravity is so strong that nothing, not even light, can escape from it. It is formed when a massive star collapses in on itself at the end of its life cycle. The gravity in a black hole is so intense that it creates a singularity, a point of infinite density at the center. Black holes have a profound impact on the surrounding space and time, distorting everything around them.


###  Enhanced Version of Pipeline

In [58]:
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

#  Define multiple subject-specific prompts
prompts = {
    "physics": """You are a very smart physics professor. \
    You excel at explaining physics concepts in a concise and easy-to-understand manner. \
    If you don't know the answer, just admit it.

    Here is a question:
    {query}""",

    "math": """You are a skilled mathematician. \
    You excel at breaking down complex problems into simpler steps. \
    Answer each component separately before solving the entire problem.

    Here is a question:
    {query}""",

    "chemistry": """You are a chemistry expert. \
    Explain chemical reactions, atomic structures, and related topics clearly.

    Here is a question:
    {query}""",

    "biology": """You are a knowledgeable biologist. \
    Explain biological concepts such as cell functions, DNA, and evolution in simple terms.

    Here is a question:
    {query}""",

    "computer_science": """You are a software engineer. \
    Answer programming-related questions, explain algorithms, and debugging techniques.

    Here is a question:
    {query}"""
}

#  Generate embeddings for each prompt
embeddings = OpenAIEmbeddings()
prompt_names = list(prompts.keys())  # ["physics", "math", "chemistry", "biology", "computer_science"]
prompt_texts = list(prompts.values())  # The actual templates
prompt_embeddings = embeddings.embed_documents(prompt_texts)  # Precomputed embeddings for routing

#  Function to find the best prompt based on semantic similarity
def prompt_router(input):
    query_embedding = embeddings.embed_query(input["query"])  # Embed the input question
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]  # Compute cosine similarity
    best_match_idx = similarity.argmax()  # Get the index of the most similar prompt

    # Multi-prompt blending: If multiple subjects are relevant, mix them!
    sorted_indices = similarity.argsort()[::-1]  # Sort similarity scores in descending order
    top_similarities = similarity[sorted_indices[:2]]  # Take top 2 matching prompts
    threshold = 0.8  # Set a threshold for similarity blending

    if len(top_similarities) > 1 and top_similarities[1] > threshold:
        chosen_prompts = [prompt_names[idx] for idx in sorted_indices[:2]]
        print(f"Blending: {chosen_prompts}")
        blended_template = f"{prompts[chosen_prompts[0]]}\n{prompts[chosen_prompts[1]]}"
        return PromptTemplate.from_template(blended_template)
    
    # Otherwise, return the best single match
    best_prompt_name = prompt_names[best_match_idx]
    print(f"Using {best_prompt_name.upper()} Prompt")
    return PromptTemplate.from_template(prompts[best_prompt_name])

#  Define the full LangChain pipeline
chain = (
    {"query": RunnablePassthrough()}  # Pass user query
    | RunnableLambda(prompt_router)  # Choose or blend prompts based on similarity
    | ChatOpenAI()  # Query the LLM with the chosen prompt
    | StrOutputParser()  # Convert the response into a clean string
)

#  Example Usage
test_questions = [
    "How does gravity affect time?",
    "What is the quadratic formula?",
    "Explain the pH scale in chemistry.",
    "How does DNA replication work?",
    "How do I reverse a linked list in Python?",
    "What happens to an electron in a magnetic field?"
]

# Run multiple test cases
for i, question in enumerate(test_questions):
    print(f"\n**User Query {i+1}:** {question}")
    response = chain.invoke( question)
    print(f"**Response:** {response}\n")



**User Query 1:** How does gravity affect time?
Using PHYSICS Prompt
**Response:** Gravity affects time by causing time dilation. According to Einstein's theory of relativity, gravity warps spacetime, causing time to move slower in stronger gravitational fields. This means that time actually passes at a different rate depending on how close you are to a massive object like a planet or star. This phenomenon has been confirmed through experiments with atomic clocks in high and low gravity environments, such as on Earth's surface compared to on the International Space Station.


**User Query 2:** What is the quadratic formula?
Using MATH Prompt
**Response:** The quadratic formula is as follows:

For a quadratic equation in the form ax^2 + bx + c = 0, the solutions can be found using the formula:

x = (-b ± √(b^2 - 4ac)) / 2a


**User Query 3:** Explain the pH scale in chemistry.
Using CHEMISTRY Prompt
**Response:** Chemical reactions are processes in which substances, called reactants, i